In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline 

In [2]:
train = pd.read_csv('../apartment/train.csv')
#train.info()

In [3]:
from sklearn import preprocessing

catagorical_cols=[]
for column in train.columns.values:
    if (train[column].dtype=='O'):
        catagorical_cols.append(column)


for x in catagorical_cols:
    train[x]=train[x].fillna('None')
    
for x in catagorical_cols:
    label = preprocessing.LabelEncoder()
    train[x] = label.fit_transform(train[x])

In [4]:
from sklearn import preprocessing
from sklearn import linear_model, svm, gaussian_process
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso
import numpy as np

In [5]:
train1 = train.drop('SalePrice',1)
X = train1
y1 =  pd.DataFrame(train['SalePrice'],columns=['SalePrice'])
y = pd.concat([ train['Id'],y1], axis=1)
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [6]:
for x in X_train.columns.values:
    if(X_train[x].isnull().sum() >0):
        X_train[x].fillna(X_train[x].mean(),inplace = True)
for y in X_test.columns.values:
    if(X_test[y].isnull().sum() >0):
        X_test[y].fillna(X_test[y].mean(),inplace = True)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [7]:
X_train2 = X_train
y_train2 = y_train['SalePrice'].values

In [8]:
minmaxscaler = preprocessing.MinMaxScaler(copy=True, feature_range=(0, 1)).fit(X_train)
X_train = X_train
y_train = y_train['SalePrice'].values
X_train = minmaxscaler.fit_transform(X_train)

X_test = X_test
y_test = y_test['SalePrice'].values
X_test = minmaxscaler.transform(X_test)

In [9]:
clfs = {
        'svm':svm.SVR(), 
        'RandomForestRegressor':RandomForestRegressor(n_estimators=200),
        'LinearRegression':linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
        'DecisionTreeRegressor':DecisionTreeRegressor(),
        'SGDRegressor':SGDRegressor(max_iter=5, tol = None),
        'GradientBoostingRegressor':GradientBoostingRegressor(),
        'MLPRegressor':MLPRegressor(solver='adam', activation = 'relu',hidden_layer_sizes=20, max_iter=1000, learning_rate='adaptive', random_state= None),
        'Lasso':Lasso(alpha = 0.1,max_iter=1000,random_state=None)
       }
for clf in clfs:
    try:
        clfs[clf].fit(X_train, y_train.ravel())   #why need to make this ravel()??
        y_pred = clfs[clf].predict(X_test)
        #print(clf + " cost:" + str(np.sum(y_pred-y_test)/len(y_pred)))
        #print(clf + " score:" + str(np.mean(np.absolute(y_pred-y_test))))
        #print(clf + " score:" + str(np.mean((y_pred-y_test)*(y_pred-y_test))))
        print(clf + " score:" + str(np.sqrt(np.mean((np.log(y_pred)-np.log(y_test))*(np.log(y_pred)-np.log(y_test))))))
    except Exception as e:
        print(clf + " Error:")
        print(str(e))

svm score:0.41183194986795385
RandomForestRegressor score:0.14088023096978228
LinearRegression score:nan
DecisionTreeRegressor score:0.20151161776467227
SGDRegressor score:0.17884522461278382


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in log


GradientBoostingRegressor score:0.13272335801688143
MLPRegressor score:2.2013912128705417
Lasso score:0.2676173864211928


D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
#prediction

In [10]:
test = pd.read_csv('../apartment/test.csv')
model = GradientBoostingRegressor()
from sklearn import preprocessing
label1 = preprocessing.LabelEncoder()

catagorical_cols1=[]
for column in test.columns.values:
    if (test[column].dtype=='O'):
        catagorical_cols1.append(column)


for x in catagorical_cols1:
    test[x]=test[x].fillna('None')
    
for x in catagorical_cols1:
    label1 = preprocessing.LabelEncoder()
    test[x] = label1.fit_transform(test[x])

In [11]:
for z in test.columns.values:
    if(test[z].isnull().sum() >0):
        test[z].fillna(test[z].mean(),inplace = True)

In [12]:
model.fit(X_train2,y_train2)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [13]:
X = test
y_predict = model.predict(X)

predicted_result = pd.DataFrame(data = y_predict,columns=['SalePrice'])
result = pd.concat([test['Id'],predicted_result],axis=1)

In [14]:
result.to_csv('../apartment/81Features.csv',index = False)